In [5]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import evaluate  
from datasets import load_dataset
import numpy as np

2025-06-16 09:07:41.578420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750064862.056564      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750064862.179406      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# 加载HuggingFace中的dataset
ds = load_dataset('PassbyGrocer/msra-ner')
ds

README.md:   0%|          | 0.00/767 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.08M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/46364 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4365 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4365 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 46364
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 4365
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 4365
    })
})

In [7]:
for items in ds['train']:
    print(items['tokens'])
    print(items['ner_tags'])
    break

['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
tokennizer = AutoTokenizer.from_pretrained('bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [9]:
# 验证tag标签数量
tags_id = set()
for items in ds['train']:
    tags_id.update(items['ner_tags'])
tags_id    


{0, 1, 2, 3, 4, 5, 6}

In [10]:
# entity_index
entites = ['O'] + list({'PER', 'LOC', 'ORG'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())
entity_index = {entity:i for i, entity in enumerate(entites)}    
    

In [11]:
tags

['O', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER']

In [12]:
entity_index

{'O': 0, 'ORG': 1, 'LOC': 2, 'PER': 3}

In [13]:
test = ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
a = tokennizer(test, 
                           truncation=True,
                           add_special_tokens=False, 
                           max_length=512, 
                           is_split_into_words=True,)
a



{'input_ids': [2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
def data_input_proc(item):
    # 文本已经分为字符，且tag索引也已经提供
    # 所以数据预处理反而简单
    # 导入已拆分为字符的列表，需要设置参数is_split_into_words=True
    input_data = tokennizer(item['tokens'], 
                           truncation=True,
                           add_special_tokens=False, 
                           max_length=512, 
                           is_split_into_words=True,
                           return_offsets_mapping=True)
    labels = [lbl[:512] for lbl in item['ner_tags']]
    input_data['labels'] = labels
    return input_data
ds1 = ds.map(data_input_proc, batched=True)    


Map:   0%|          | 0/46364 [00:00<?, ? examples/s]

Map:   0%|          | 0/4365 [00:00<?, ? examples/s]

Map:   0%|          | 0/4365 [00:00<?, ? examples/s]

In [15]:
ds1.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [16]:
for item in ds1['train']:
    print(item)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])}


In [17]:

id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}
model = AutoModelForTokenClassification.from_pretrained('bert-base-chinese', 
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [18]:
args = TrainingArguments(
    output_dir="msra_ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    # save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

In [19]:
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts, axis=2)
    
    # 准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results

In [20]:

data_collator = DataCollatorForTokenClassification(tokenizer=tokennizer, padding=True)

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds1['train'],
    eval_dataset=ds1['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [22]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.061500,0.030118,"{'precision': 0.8578607322325915, 'recall': 0.8984962406015038, 'f1': 0.8777084098420859, 'number': 1330}","{'precision': 0.932937478169752, 'recall': 0.9290434782608695, 'f1': 0.9309864064133844, 'number': 2875}","{'precision': 0.8201928530913216, 'recall': 0.8854868340477648, 'f1': 0.8515901060070672, 'number': 1633}",0.882539,0.909901,0.896011,0.990805
2,0.019500,0.028727,"{'precision': 0.8747300215982722, 'recall': 0.9135338345864662, 'f1': 0.8937109231335051, 'number': 1330}","{'precision': 0.9562764456981664, 'recall': 0.943304347826087, 'f1': 0.9497461040098056, 'number': 2875}","{'precision': 0.9253731343283582, 'recall': 0.9491733006736068, 'f1': 0.9371221281741233, 'number': 1633}",0.928305,0.938164,0.933208,0.992787
3,0.006500,0.031377,"{'precision': 0.8692253020611229, 'recall': 0.9195488721804511, 'f1': 0.8936792108147605, 'number': 1330}","{'precision': 0.956811797752809, 'recall': 0.9478260869565217, 'f1': 0.9522977459374453, 'number': 2875}","{'precision': 0.9409975669099757, 'recall': 0.9473361910593999, 'f1': 0.944156240463839, 'number': 1633}",0.931514,0.941247,0.936355,0.992834


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Trainer is attempting to log a value of "{'precision': 0.8578607322325915, 'recall': 0.8984962406015038, 'f1': 0.8777084098420859, 'number': 1330}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.932937478169752, 'recall': 0.9290434782608695, 'f1': 0.9309864064133844, 'number': 2875}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8201928530913216, 'recall': 0.8854868340477648, 'f1': 0.8515901060070672, 'number': 1633}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to ga

TrainOutput(global_step=2175, training_loss=0.02357312038027007, metrics={'train_runtime': 2157.9292, 'train_samples_per_second': 64.456, 'train_steps_per_second': 1.008, 'total_flos': 1.1644638621676104e+16, 'train_loss': 0.02357312038027007, 'epoch': 3.0})

In [24]:
from transformers import pipeline
pipeline = pipeline('token-classification', 'msra_ner_train/checkpoint-2175')
pipeline('我爱北京天安门')

Device set to use cuda:0


[{'entity': 'B-ORG',
  'score': 0.9889332,
  'index': 3,
  'word': '北',
  'start': 2,
  'end': 3},
 {'entity': 'I-ORG',
  'score': 0.99495465,
  'index': 4,
  'word': '京',
  'start': 3,
  'end': 4},
 {'entity': 'B-ORG',
  'score': 0.99496955,
  'index': 5,
  'word': '天',
  'start': 4,
  'end': 5},
 {'entity': 'I-ORG',
  'score': 0.9896348,
  'index': 6,
  'word': '安',
  'start': 5,
  'end': 6},
 {'entity': 'I-ORG',
  'score': 0.9926347,
  'index': 7,
  'word': '门',
  'start': 6,
  'end': 7}]